In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, log_loss
import matplotlib.pyplot as plt

In [16]:
file_path = "data/voice.csv"  
data = pd.read_csv(file_path)

data['label'] = data['label'].map({'male': 1, 'female': 0})

X = data.drop(columns=['label']).values  
y = data['label'].values  

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)

In [18]:
X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_test = (X_test - X_test.mean(axis=0)) / X_test.std(axis=0)

In [19]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return np.where(z > 0, 1, 0)

def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))

def binary_cross_entropy_derivative(y_true, y_pred):
    return (y_pred - y_true) / (y_pred * (1 - y_pred) + 1e-8)

In [20]:
def initialisation(n0, n1, n2):
    W1 = np.random.randn(n1,n0)
    b1 = np.random.randn(n1,1)
    W2 = np.random.randn(n2,n1)
    b2 = np.random.randn(n2,1)

    parameter = {'W1': W1,
                 'b1': b1,
                 'W2':W2,
                 'b2':b2}
    
    return parameter


In [21]:
def forward_propagation(X, parameter):
    W1 = parameter['W1']
    W2 = parameter['W2']
    b1 = parameter['b1']
    b2 = parameter['b2']

    Z1 = W1.dot(X)+b1
    A1 = relu(Z1)

    Z2 = W2.dot(A1)+b2
    A2 = sigmoid(Z2)
    
    activations = {'A1':A1,
                   'A2':A2}

    return activations

In [22]:
def back_propragation(X, y, parameter, activations):
    A1 = activations['A1']
    A2 = activations['A2']
    W2 = parameter['W2']

    m = y.shape[1]

    dZ2 = A2 - y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2, axis = 1, keepdims=True)

    dZ1 = np.dot(W2.T, dZ2)*A1*(1-A1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)

    gradients = {'db1': db1,
                 'db2': db2,
                 'dW1': dW1,
                 'dW2': dW2}
    return gradients


In [23]:
def update(gradients, parameter, learning_rate):

    dW1 = gradients['dW1']
    dW2 = gradients['dW2']
    db1 = gradients['db1']
    db2 = gradients['db2']

    W1 = parameter['W1']
    W2 = parameter['W2']
    b1 = parameter['b1']
    b2 = parameter['b2']


    W1 = W1 - learning_rate *dW1
    W2 = W2 - learning_rate *dW2
    b1 = b1 - learning_rate *db1
    b2 = b2 - learning_rate *db2

    parameter = {'W1':W1,
                 'W2':W2,
                 'b1':b1,
                 'b2':b2}
    
    return parameter

In [24]:
def predict(X, parameter):
    activations = forward_propagation(X, parameter)
    A2 = activations['A2']
    return A2 >=0.5

In [26]:
def neural_network(X, y, n1=16, learning_rate = 0.1, n_iter = 1000):

    # initialisation parametres
    n0 = X.shape[0]
    n2 = y.shape[0]
    np.random.seed(0)
    parametres = initialisation(n0, n1, n2)

    train_loss = []
    train_acc = []
    history = []

    # gradient descent
    for i in tqdm(range(n_iter)):
        activations = forward_propagation(X, parametres)
        A2 = activations['A2']

        # Plot courbe d'apprentissage
        train_loss.append(log_loss(y.flatten(), A2.flatten()))
        y_pred = predict(X, parametres)
        train_acc.append(accuracy_score(y.flatten(), y_pred.flatten()))
        
        history.append([parametres.copy(), train_loss, train_acc, i])

                # mise a jour
        gradients = back_propragation(X, y, parametres, activations)
        parametres = update(gradients, parametres, learning_rate)


    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='train loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='train acc')
    plt.legend()
    plt.show()

    return parametres

In [30]:
X = X_train.T
y = y_train.reshape((1, y_train.shape[0]))

In [33]:
print('dimensions de X:', X.shape)
print('dimensions de y:', y.shape)


dimensions de X: (20, 2534)
dimensions de y: (1, 2534)


In [34]:
neural_network(X, y, n1=32)

  0%|          | 1/1000 [00:00<06:54,  2.41it/s]C:\Users\Antoine\AppData\Local\Temp\ipykernel_72028\2254101342.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\Antoine\AppData\Local\Temp\ipykernel_72028\2254101342.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\Antoine\AppData\Local\Temp\ipykernel_72028\2254101342.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\Antoine\AppData\Local\Temp\ipykernel_72028\2254101342.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
  0%|          | 3/1000 [00:00<02:48,  5.92it/s]C:\Users\Antoine\AppData\Local\Temp\ipykernel_72028\2254101342.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\Antoine\AppData\Local\Temp\ipykernel_72028\2254101342.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\Antoine\AppData\Local\Temp\ipykernel_

ValueError: Input contains NaN.